# 2023 AI/ML UnicornGym - Track 1

@jesamkim | 2023-Feb-08

# 6. 훈련 파이프라인

### setup environment

In [1]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

In [2]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()


%store -r 
# 노트북에 저장되어 있는 변수를 보기 위해서는 주석을 제거하고 실행하시면 됩니다.
%store  

Stored variables and their in-db values:
bucket                      -> 'sagemaker-us-east-1-376278017302'
endpoint_name               -> 'sagemaker-xgboost-2023-02-08-05-34-18-143'
hyperparameters             -> {'scale_pos_weight': '19', 'max_depth': '2', 'eta'
input_data_uri              -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
project_prefix              -> 'titanic-ml'
s3_path                     -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
test_data_uri               -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
test_df                     ->     PassengerId  Pclass  Sex   Age  SibSp  Parch  
train_data_uri              -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/


---
## 6-1. 훈련에 사용할 전처리된 파일을 확인
이후에 훈련에서 사용할 S3의 저장된 전처리 데이터를 확인 합니다.

### 데이터 세트 로딩
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
- 로딩힐 데이터 파일이 S3에 있는지 변수의 경로를 확인 합니다. (train_preproc_dir_artifact)

In [3]:
train_preproc_dir_artifact = input_data_uri 

In [4]:
train_preproc_dir_artifact_file = os.path.join(train_preproc_dir_artifact,'train.csv')
print("train_preproc_dir_artifact_file: \n", train_preproc_dir_artifact_file)
train_prep_df = pd.read_csv(train_preproc_dir_artifact_file)

train_prep_df = train_prep_df.drop('Unnamed: 0', axis=1)

train_prep_df

train_preproc_dir_artifact_file: 
 s3://sagemaker-us-east-1-376278017302/titanic-ml/input/train.csv


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,1,1,38.0,1,0,71.2833,1
1,1,1,1,35.0,1,0,53.1000,0
2,0,1,0,54.0,0,0,51.8625,0
3,1,3,1,4.0,1,1,16.7000,0
4,1,1,1,58.0,0,0,26.5500,0
...,...,...,...,...,...,...,...,...
178,1,1,1,47.0,1,1,52.5542,0
179,0,1,0,33.0,0,0,5.0000,0
180,1,1,1,56.0,0,1,83.1583,1
181,1,1,1,19.0,0,0,30.0000,0


---
## 6-2. 모델 빌딩 파이프라인의 스텝(Step) 생성

### 모델 빌딩 파이프라인 변수 생성

파이프라인에서 사용할 파이프라인 파라미터를 정의합니다. 파이프라인을 스케줄하고 실행할 때 파라미터를 이용하여 실행조건을 커스마이징할 수 있습니다. 파라미터를 이용하면 파이프라인 실행시마다 매번 파이프라인 정의를 수정하지 않아도 됩니다.



In [5]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value= 1
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


---
## 6-3. 하이퍼파라미터 세팅

### 기본 훈련 변수 및 하이퍼파라미터 설정
- XGBoost에 알고리즘에 입력될 하이퍼 파리미터의 값을 설정 합니다.
- scale_pos_weight 의 경우는 현재의 데이터가 레이블(fraud)간 불균형이 있기에, fraud: 1, non-fraud: 0 의 비율을 계산하여 제공합니다.
- 하이퍼 파라미터 상세 사항은 여기를 보세요. -->  [XGBoost Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html)

In [6]:
from sagemaker.xgboost.estimator import XGBoost


estimator_output_path = f's3://{bucket}/{project_prefix}/training_jobs'
train_instance_count = 1

hyperparameters = {
       "scale_pos_weight" : "19",    
        "max_depth": "2",
        "eta": "0.3",
        "objective": "binary:logistic",
        "num_round": "100",
}
%store hyperparameters

Stored 'hyperparameters' (dict)


---
## 6-4. Estimator 생성

### 모델 학습을 위한 학습단계 정의 

 XGBoost 사용자 알고리즘을 이용하도록 Estimator를 구성합니다. 
 - 보편적인 학습스크립트를 이용하여 입력 채널에서 정의한 학습데이터를 로드하고, 하이퍼파라미터 설정을 통해 학습을 설정하고, 모델을 학습한 후 `model_dir`경로에 학습된 모델을 저장합니다. 저장된 모델은 이후 호스팅을 위해 사용됩니다. 
- 사용자 훈련 코드 "src/xgboost_starter_script_pipeline.py"
- 훈련이 끝난 후에 결과인 모델 아티펙트를 경로 "estimator_output_path" 에 지정 합니다. 지정 안할 시에는 디폴트 경로로 저장 됩니다.


- 파이프라인 파라미터인 `training_instance_type`, `training_instance_count` 파라미터가 사용된 것을 확인합니다. 

In [7]:
xgb_train = XGBoost(
    entry_point = "xgboost_starter_script_pipeline.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = training_instance_count,
    instance_type = training_instance_type,
    framework_version="1.3-1"    
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is not allowed. The default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


---
## 6-5. 모델 훈련 스탭 생성
- 스텝 생성시에 위에서 생성한 Estimator 입력 및 입력 데이타로서 전처리 데이터가 존재하는 S3 경로를 제공합니다.

In [8]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="Survived-Basic-Train",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data= train_preproc_dir_artifact,
            content_type="text/csv"
        ),
    },
)

---
## 6-6. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행




### 파이프라인 정의
파이프라인 정의시에 아래 3개의 인자를 제공합니다.
- 파이프라인 이름
- 파이프라인 파라미터
- 파이프라인 실험 설정
- 스텝 정의 (여기서는 훈련, 모델 등록 두가지 스텝 임)

In [9]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        training_instance_type,        
        training_instance_count,         
        input_data,
    ],
    steps=[step_train],
)

### 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [10]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'TrainingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-376278017302/titanic-ml/input'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Survived-Basic-Train',
   'Type': 'Training',
   'Arguments': {'AlgorithmSpecification': {'TrainingInputMode': 'File',
     'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1'},
    'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-376278017302/titanic-ml/training_jobs'},
    'StoppingCondition': {'MaxRuntimeInSeconds': 86400},
    'ResourceConfig': {'VolumeSizeInGB': 30,
     'InstanceCount': {'Get': 'Parameters.TrainingInstanceCount'},

---
## 6-7. 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [11]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [12]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:376278017302:pipeline/titanic-ml',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:376278017302:pipeline/titanic-ml/execution/354rt2nvtof4',
 'PipelineExecutionDisplayName': 'execution-1675850137533',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2023, 2, 8, 9, 55, 37, 454000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 2, 8, 9, 55, 37, 454000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'e0e3cdfa-e8a0-4bc1-9e01-c10f1f15864b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e0e3cdfa-e8a0-4bc1-9e01-c10f1f15864b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '385',
   'date': 'Wed, 08 Feb 2023 09:55:36 GMT'},
  'RetryAttempts': 0}}

---
## 6-8. 파이프라인 실행 기다리기

In [13]:
execution.wait()

실행이 완료될 때까지 기다립니다.

---
## 6-9. 파이프라인 실행 단계 기록 보기

In [14]:
execution.list_steps()

[{'StepName': 'Survived-Basic-Train',
  'StartTime': datetime.datetime(2023, 2, 8, 9, 55, 38, 819000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 2, 8, 9, 58, 2, 37000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:376278017302:training-job/pipelines-354rt2nvtof4-Survived-Basic-Train-QHhe2K0bjJ'}}}]

---
## 6-10. 세이지 메이커 스튜디오에서 확인하기
- 아래의 그림 처럼 SageMaker Studio에 로긴후에 따라하시면, SageMaker Studio 에서도 실행 내역을 확인할 수 있습니다.
    - SageMaker Studio 개발자 가이드 --> [SageMaker Studio](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/studio.html)

![train_step_sm.studio.png](img/train_step_sm.studio.png)

---
## 6-11. 모델 아티펙트 경로 추출

### 훈련 모델 결과의 경로 지정
- 다음 노트북에서 사용할 훈련 모델 아티펙트의 S3 경로를 저장 합니다.

In [16]:
from src.p_utils import get_train_artifact

import boto3
client = boto3.client("sagemaker")
    
train_model_artifact = get_train_artifact(execution, client,job_type='TrainingJob', kind=0)
print(" train_model_artifact: \n", train_model_artifact)


 train_model_artifact: 
 s3://sagemaker-us-east-1-376278017302/titanic-ml/training_jobs/pipelines-354rt2nvtof4-Survived-Basic-Train-QHhe2K0bjJ/output/model.tar.gz


### 훈련시 사용한 도커 이미지를 저장
- 추론시에 사용할 도커 이미지를 추론시에도 동일하게 사용하기 위해서 저장 합니다.
    - 일반적으로 훈련, 추론의 도커 이미지를 분리해서 사용합니다. 하지만 간단한 알고리즘의 경우는 훈련, 추론의 도커 이미지를 같이 사용하기도 합니다.

In [17]:
image_uri = xgb_train.image_uri
print("image_uri: \n", image_uri)

image_uri: 
 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1


훈련 모델 아티펙트와, 훈련시 사용한 다커 이미지의 경로를 저장 합니다.

In [18]:
%store train_model_artifact
%store image_uri

Stored 'train_model_artifact' (str)
Stored 'image_uri' (str)
